# 0. SOME PRELIMINARIES 

In [20]:
# Import some libraries
import matplotlib.pyplot as plt 
# For plotting data
import numpy as np              
# For Panda dataframes. A dataframe is a matrix-like structure, 
# similar to R dataframes  
import pandas as pd

import os
os.getcwd()

'C:\\Users\\Clara\\Documents'

The "wind_pickle" file contains data in a binary format called "Pickle". Pickle data loads faster than text data.

In [21]:
data = pd.read_pickle('wind_pickle')

You can visualize the attributes in the dataset. Very important, the output attribute (i.e. the value to be predicted, **energy**, is the first attribute). **Steps** represents the hours in advance of the forecast. We will not use this variable here.

In [23]:
# The dataset contains 5937 instances and 556 attributes (including 
# the outcome to be predicted)
print(data.shape)
data.columns.values.tolist()

(5937, 556)


['energy',
 'steps',
 'year',
 'month',
 'day',
 'hour',
 'p54.162.1',
 'p54.162.2',
 'p54.162.3',
 'p54.162.4',
 'p54.162.5',
 'p54.162.6',
 'p54.162.7',
 'p54.162.8',
 'p54.162.9',
 'p54.162.10',
 'p54.162.11',
 'p54.162.12',
 'p54.162.13',
 'p54.162.14',
 'p54.162.15',
 'p54.162.16',
 'p54.162.17',
 'p54.162.18',
 'p54.162.19',
 'p54.162.20',
 'p54.162.21',
 'p54.162.22',
 'p54.162.23',
 'p54.162.24',
 'p54.162.25',
 'p55.162.1',
 'p55.162.2',
 'p55.162.3',
 'p55.162.4',
 'p55.162.5',
 'p55.162.6',
 'p55.162.7',
 'p55.162.8',
 'p55.162.9',
 'p55.162.10',
 'p55.162.11',
 'p55.162.12',
 'p55.162.13',
 'p55.162.14',
 'p55.162.15',
 'p55.162.16',
 'p55.162.17',
 'p55.162.18',
 'p55.162.19',
 'p55.162.20',
 'p55.162.21',
 'p55.162.22',
 'p55.162.23',
 'p55.162.24',
 'p55.162.25',
 'cape.1',
 'cape.2',
 'cape.3',
 'cape.4',
 'cape.5',
 'cape.6',
 'cape.7',
 'cape.8',
 'cape.9',
 'cape.10',
 'cape.11',
 'cape.12',
 'cape.13',
 'cape.14',
 'cape.15',
 'cape.16',
 'cape.17',
 'cape.18',
 'ca

Below, data is going to be separated in train, validation, and test. Given that the use of Pandas dataframes is quite advanced, and doing this for you:

In [24]:
indicesTrain = (np.where(data.year<=2006))[0]
indicesVal = (np.where((data.year==2007) | (data.year==2008)))[0]
indicesTest = (np.where(data.year>=2009))[0]

Beware!, **indicesTrain** does not contain the training data, but the *indices* of the training data. For instance, the following cell means that training data is made of instance number 0, instance number 1, ..., up to instance number 2527. This will be important later.

In [25]:
indicesTrain

array([   0,    1,    2, ..., 2525, 2526, 2527], dtype=int64)

Now, we are going to transform **data**, which is a Pandas dataframe, to **ava**, which is a NumPy matrix. The reason is that Scikit-learn uses NumPy matrices, not Panda dataframes.

In [26]:
ava = data.as_matrix()

Now, **ava** is going to be decomposed into inputs **X** and outputs **y**. And then, into training, validation, and test. For instance, **Xava** and **yava** contain the input attributes, and the output attribute (**energy**) of the whole dataset. Please, ask yourself why the inputs use "6:" and the output use "0". **Xtrain** and **ytrain** are the same, but for the training dataset.

In [61]:
Xava = ava[:,6:]; yava = ava[:,0]
Xtrain = ava[indicesTrain,6:]; ytrain = ava[indicesTrain,0]
Xval = ava[indicesVal,6:]; yval = ava[indicesVal,0]
Xtest = ava[indicesTest,6:][:,6:]; ytest = ava[indicesTest,0]

(1299L, 550L)

The following cell defines function **mae** (Mean Absolute Error), that we will use later to measure the accuracy of models.

In [28]:
def mae(yval_pred, yval):
  val_mae = metrics.mean_absolute_error(yval_pred, yval)
  return(val_mae)

The following cell trains KNN with (Xtrain, ytrain) and evaluates it with (Xval, yval).

In [29]:
from sklearn import metrics
from sklearn import neighbors
n_neighbors = 5
knn = neighbors.KNeighborsRegressor(n_neighbors, weights='uniform')
np.random.seed(0)
%time _ = knn.fit(Xtrain, ytrain)
yval_pred = knn.predict(Xval)

print "MAE for KNN with K=5 is {}".format(mae(yval_pred, yval))

Wall time: 80 ms
MAE for KNN with K=5 is 486.911414935


In [30]:
# In case you need help for KNN
help('sklearn.neighbors.KNeighborsRegressor')

Help on class KNeighborsRegressor in sklearn.neighbors:

sklearn.neighbors.KNeighborsRegressor = class KNeighborsRegressor(sklearn.neighbors.base.NeighborsBase, sklearn.neighbors.base.KNeighborsMixin, sklearn.neighbors.base.SupervisedFloatMixin, sklearn.base.RegressorMixin)
 |  Regression based on k-nearest neighbors.
 |  
 |  The target is predicted by local interpolation of the targets
 |  associated of the nearest neighbors in the training set.
 |  
 |  Read more in the :ref:`User Guide <regression>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, optional (default = 5)
 |      Number of neighbors to use by default for :meth:`k_neighbors` queries.
 |  
 |  weights : str or callable
 |      weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of their distance.
 |        in this case, closer neighbors of a query p

The following cell, does hyper-parameter tuning for parameter K (n_neighbors), from 1 to 4 by 1. Please, notice that with **partitions = [(indicesTrain, indicesVal)]** we are telling **gridSearch** to use the training dataset for training the different models with the different parameters, and the validation dataset for testing. Notice that this is different to other notebooks, where crossvalidation was used for this purpose. 

In [31]:
from sklearn.grid_search import GridSearchCV
np.random.seed(0)
param_grid = {'n_neighbors': range(1,4,1)}

partitions = [(indicesTrain, indicesVal)]
clf = GridSearchCV(neighbors.KNeighborsRegressor(), 
                   param_grid,
                   scoring='mean_absolute_error',
                   cv=partitions , verbose=1)
%time _ = clf.fit(Xava,yava)

Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s finished


Wall time: 1.86 s


Next, we show the best K parameter and the MAE of the final model built with the best parameter.

In [32]:
print "Best K: {} and MAE for best K: {}".format(clf.best_params_, -clf.best_score_)

Best K: {'n_neighbors': 3} and MAE for best K: 503.711691044


# 1. HOW LONG DOES IT TAKE?

It is always a good idea to have some estimation of how long your machine learning algorithm is going to take. In the next two cells, try to estimate how many seconds KNN (with K=3) does it take, with only **100 instances**. With 6000 instances, it will take approximately 60 times that number. You can use **%time** for timing, as in previous cells.

In [33]:
#<WRITE CODE HERE FOR TIMING KNN>

n_neighbors = 3
Xtrain_100 = ava[:100,6:]; ytrain_100 = ava[:100,0]
knn = neighbors.KNeighborsRegressor(n_neighbors)

np.random.seed(0)
%time _ = knn.fit(Xtrain_100, ytrain_100)

Wall time: 4 ms


Please, do the same for Decision trees with default parameters

In [34]:
#<WRITE CODE HERE FOR TIMING Decision Trees>

from sklearn import tree

reg = tree.DecisionTreeRegressor()

np.random.seed(0)
%time _ = reg.fit(Xtrain_100, ytrain_100)

Wall time: 44 ms


# 2. MODEL SELECTION AND HYPER-PARAMETER TUNING

Train a KNN model with default parameters

In [35]:
#<WRITE CODE HERE FOR KNN>

knn_def = neighbors.KNeighborsRegressor()

np.random.seed(0)
%time _ = knn_def.fit(Xtrain, ytrain)


yval_pred_knn_def = knn_def.predict(Xval)

print "MAE for KNN with K={} (default) is {}".format(knn_def.n_neighbors, mae(yval_pred_knn_def, yval))

Wall time: 88 ms
MAE for KNN with K=5 (default) is 486.911414935


Do hyper-parameter tuning for KNN. Can you improve results? Note: if **gridSearch** takes too long, you can use **Randomized Search** instead.

In [36]:
#<WRITE CODE HERE FOR KNN>

param_grid_knn = {
    'n_neighbors': range(1,20,1),
    'weights' : ['uniform', 'distance']   
}

partitions = [(indicesTrain, indicesVal)]
knn_grid = GridSearchCV(neighbors.KNeighborsRegressor(), 
                   param_grid_knn,
                   scoring='mean_absolute_error',
                   cv=partitions , verbose=1)

knn_grid.fit(Xava, yava)

print("The optimal combination of parameters obtained with Grid Search is: " + str(knn_grid.best_params_) + " with " + str(-knn_grid.best_score_) + " MAE.")

Fitting 1 folds for each of 38 candidates, totalling 38 fits


[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   23.0s finished


The optimal combination of parameters obtained with Grid Search is: {'n_neighbors': 17, 'weights': 'distance'} with 469.147029647 MAE.


We do improve the results, because the Mean Absolute Error is reduced.
* We obtained a MAE = 486.911414935 with K=5 (default).
* We obtain a MAE = 469.147029647 with K=17 (optimal parameter obtained with Grid Search).

Train a decision tree for regression with default parameters

In [37]:
#<WRITE CODE HERE FOR DECISION TREES>

reg_def = tree.DecisionTreeRegressor()

np.random.seed(0)
%time _ = reg_def.fit(Xtrain, ytrain)

yval_pred_reg_def = reg_def.predict(Xval)

print "MAE for decision tree regressor with max_depth={} and min_samples_leaf={} (default) is {}".format(reg_def.max_depth, reg_def.min_samples_leaf, mae(yval_pred_knn_def, yval))

Wall time: 2.2 s
MAE for decision tree regressor with max_depth=None and min_samples_leaf=1 (default) is 486.911414935


Do hyper-parameter tuning for Decision trees. Can you improve results?

In [39]:
#<WRITE CODE HERE FOR DECISION TREES>

param_grid_reg = {
    'max_depth': range(1,10,1),
    'min_samples_leaf': range(1,10,1) 
}

partitions = [(indicesTrain, indicesVal)]
reg_grid = GridSearchCV(tree.DecisionTreeRegressor(), 
                   param_grid_reg,
                   scoring='mean_absolute_error',
                   cv=partitions , verbose=1)

reg_grid.fit(Xava, yava)

print("The optimal value of max_depth and min_samples_leaf obtained with Grid Search is: " + str(reg_grid.best_params_) + " with " + str(-reg_grid.best_score_) + " MAE.")

Fitting 1 folds for each of 81 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   29.5s
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  1.1min finished


The optimal value of max_depth and min_samples_leaf obtained with Grid Search is: {'max_depth': 5, 'min_samples_leaf': 6} with 303.111990619 MAE.


Train a Random Forest (RF) with default parameters. A RF is an ensemble technique based on Decision Trees, but instead of training just a single decision tree, it trains many of them and then computes the average of the outputs. Please, bear in mind that a RF with default parameters involves training 100 trees. You can estimate by hand how long it is going to take, and if it is excessive, you can lower the number of decision trees in the ensemble. 

In [40]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

# help('sklearn.ensemble.RandomForestRegressor')
#<WRITE CODE HERE FOR RANDOM FORESTS>

%time _ = rf.fit(Xtrain, ytrain)

yval_pred_rf = rf.predict(Xval)

print "MAE for Random Forest with n_estimators={} (default) is {}".format(rf.n_estimators, mae(yval_pred_rf, yval))

Wall time: 13.6 s
MAE for Random Forest with n_estimators=10 (default) is 291.774380293


Do hyper-parameter tuning for Random Forests. Their main hyper-parameter is **n_estimators**, which is the number of decision trees in the ensemble. Check some values around the default value (like, 50, 100, 150, ...). Please, bear in mind this is going to take time ... In case you want to use other hyper-parameters, please ask the teacher.

In [54]:
#<WRITE CODE HERE FOR RANDOM FORESTS HYPER-PARAMETER TUNING>

# The default value of n_estimators is 10, so we try 10 values around that number, i.e. from 5 to 15. 

param_grid_rf = {'n_estimators': range(5, 15)}

partitions = [(indicesTrain, indicesVal)]
rf_grid = GridSearchCV(RandomForestRegressor(), 
                   param_grid_rf,
                   scoring='mean_absolute_error',
                   cv=partitions , verbose=1)

rf_grid.fit(Xava, yava)

print("The optimal value of n_estimators obtained with Grid Search is: " + str(rf_grid.best_params_) + " with " + str(-rf_grid.best_score_) + " MAE.")

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.5min finished


The optimal value of n_estimators obtained with Grid Search is: {'n_estimators': 13} with 284.178279742 MAE.


Train a Gradient Tree Boosting (GB) with default parameters. A GB is also an ensemble technique based on Decision Trees. In this case, the second decision tree tries to fix the mistakes of the first decision tree. The third decision tree tries to fix the mistakes of the first two decision trees. An so on.

Please, bear in mind that a GB with default parameters involves training 100 trees. You can estimate by hand how long it is going to take, and if it is excessive, you can lower the number of decision trees in the ensemble. 

In [42]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
# help('sklearn.ensemble.GradientBoostingRegressor')
#<WRITE CODE HERE FOR GRADIENT BOOSTING>

%time _ = gb.fit(Xtrain, ytrain)

yval_pred_gb = gb.predict(Xval)

print "MAE for Gradient Tree Boosting with n_estimators={} (default) is {}".format(gb.n_estimators, mae(yval_pred_gb, yval))

Wall time: 19.6 s
MAE for Gradient Tree Boosting with n_estimators=100 (default) is 279.850958585


Do hyper-parameter tuning for Gradient Boosting. Their main hyper-parameter is **n_estimators**, which is the number of decision trees in the ensemble. Check some values around the default value (like, 50, 100, 150, ...). Please, bear in mind this is going to take time ... In case you want to use other hyper-parameters, please ask the teacher.

In [48]:
#<WRITE CODE HERE FOR GRADIENT BOOSTING HYPER-PARAMETER TUNING>

#param_grid_gb = {'n_estimators': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]}
param_grid_gb = {'n_estimators': range(50, 150, 10)}

partitions = [(indicesTrain, indicesVal)]
gb_grid = GridSearchCV(GradientBoostingRegressor(), 
                   param_grid_gb,
                   scoring='mean_absolute_error',
                   cv=partitions , verbose=1)

gb_grid.fit(Xava, yava)

print("The optimal value of n_estimators obtained with Grid Search is: " + str(gb_grid.best_params_) + " with " + str(-gb_grid.best_score_) + " MAE.")

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.0min finished


The optimal value of n_estimators obtained with Grid Search is: {'n_estimators': 140} with 278.345845545 MAE.


At this point, you should know which model performs best, and what hyper-parameters to use. Please, evaluate that best performing model on the test set.

## Results

* MAE for KNN with K=5 (default) is 486.911414935
* MAE for KNN with K=17 and weights=distance is 469.147029647


* MAE for decision tree regressor with max_depth=None and min_samples_leaf=1 (default) is 486.911414935
* MAE for decision tree regressor with max_depth=5 and min_samples_leaff=6 is 303.111990619


* MAE for Random Forest with n_estimators=10 (default) is 291.774380293
* MAE for Random Forest with n_estimators=13 is 284.178279742


* MAE for Gradient Tree Boosting with n_estimators=100 (default) is 279.850958585
* MAE for Gradient Tree Boosting with n_estimators=140 is 278.345845545  ---> BEST.



In [56]:
#<WRITE CODE HERE FOR BEST MODEL EVALUATION ON THE TEST SET>

gb_final = GradientBoostingRegressor(n_estimators=140)

gb_final.fit(Xtrain, ytrain)

ytest_pred_gb = gb_final.predict(Xtest)

print "MAE for Gradient Tree Boosting with n_estimators={} is {}".format(gb_final.n_estimators, mae(ytest_pred_gb, ytest))

ValueError: Number of features of the model must  match the input. Model n_features is 550 and  input n_features is 544 

# 3. ATTRIBUTE SELECTION

This section is more open-ended than the previous ones, and I offer less guidance. It is definitely harder, but you can always ask the teacher. 

You have to answer the following question: 

- "Are all 550 input attributes actually necessary in order to get a good model? Is it possible to have an accurate model that uses fewer than 550 variables? How many? Is it enough to have the attributes for the actual Sotavento location? (13th in the grid)"

In order to answer this question:

1) Go through the "Attribute Selection" ipython notebook, and understand the main ideas about **SelectKBest** and **Pipeline**.

2) Use **SelectKBest** and **Pipeline** (and whatever else you need) in order to find a subset of attributes that allows to build an accurate Decision Tree model. We are going to use here Decision Trees because they are faster (even if Random Forests or Gradient Boosting performed better in previous sections). Please, note that you cannot just copy/paste from the "Attribute Selection" notebook. You will have to think about how to use the main ideas from that notebook, and change whatever needs changing. 

3) Once you have decided which attributes should be used for the Decision Tree, evaluate the final model on the test dataset.


In [24]:
#<USE AS MANY CELLS AS YOU NEED>

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

param_grid = {'feature_selection__k': range(1, Xtrain.shape[1]+1),
              'regression__max_depth': range(1, 10)}

clf = Pipeline([
  ('feature_selection', SelectKBest(f_regression)),
  ('regression', tree.DecisionTreeRegressor())
])

partitions = [(indicesTrain, indicesVal)]
clf_grid = GridSearchCV(clf, 
                        param_grid,
                        scoring='mean_absolute_error',
                        cv=partitions , n_jobs=1, verbose=1)

clf_grid.fit(Xava, yava) 

print("The optimal value of max_depth obtained with Grid Search is: " + str(clf_grid.best_params_) + " with " + str(-clf_grid.best_score_) + " MAE.")

Fitting 1 folds for each of 4950 candidates, totalling 4950 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    4.5s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   20.4s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   52.6s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.8min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  3.4min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  5.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  9.6min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed: 14.8min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed: 22.0min
[Parallel(n_jobs=1)]: Done 4950 out of 4950 | elapsed: 31.1min finished


The optimal value of max_depth obtained with Grid Search is: {'feature_selection__k': 536, 'regression__max_depth': 5} with 309.557283099 MAE.
